In [ ]:
'''
This piece of code does the following things: 
1. implements fgbgmasks and Knn masks around wrists 
2. gaussian blur and techniques are used to get contours of the rope - works well 
3. regions of interests are shown with the same contours 

'''
path = r"C:\Users\ayush\Desktop\Jump Rope Code\Media\JumpRopeChairAngle_Trim.mp4"
path1 = r"C:\Users\ayush\Desktop\Jump Rope Code\Media\housejumpcut.mp4"t



import cv2
import mediapipe as mp
import numpy as np 

# Load the video
cap = cv2.VideoCapture(path)

#masks
fgbg = cv2.createBackgroundSubtractorMOG2()
knnfgbg = cv2.createBackgroundSubtractorKNN()


anglecount = []
# Initialize the OpenPose model
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:

    while cap.isOpened():
        # Read the frame
        ret, frame = cap.read()
        if not ret:
            break

        # Convert the frame to RGB
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        # Process the frame with OpenPose to get the landmarks
        results = pose.process(frame_rgb)
        right_wrist = results.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_WRIST]
        left_wrist = results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_WRIST]
        
    
        
        
        # If left wrist landmark is detected, draw the bounding rectangle around it
        if left_wrist.visibility > 0:
            left_wrist_x = int(left_wrist.x * frame.shape[1])
            left_wrist_y = int(left_wrist.y * frame.shape[0])
            cv2.rectangle(frame, (left_wrist_x - 30, left_wrist_y - 30),
                          (left_wrist_x + 40, left_wrist_y + 30), (0, 255, 0), 2)
            try:
                roi1 = frame[left_wrist_y - 30:left_wrist_y + 30, left_wrist_x - 30:left_wrist_x + 40]
            except : 
                roi1 = frame[200:280,200:280]
                
                
        ##If right wrist landmark is detected, draw the bounding rectangle around it
        if right_wrist.visibility > 0:
            right_wrist_x = int(right_wrist.x * frame.shape[1])
            right_wrist_y = int(right_wrist.y * frame.shape[0])
            cv2.rectangle(frame, (right_wrist_x - 40, right_wrist_y - 30),
                          (right_wrist_x + 30 , right_wrist_y + 30), (0, 255, 0), 2)
            
            roi2 = frame[right_wrist_y - 30:right_wrist_y + 30, right_wrist_x - 30:right_wrist_x + 30]

        #converting video to graya and applying filters to make the edge detection more inline with our requiremnts
        gray = cv2.cvtColor(roi2,cv2.COLOR_BGR2GRAY)
        blur = cv2.GaussianBlur(gray,(5,5),0)
        ret, thresh_img = cv2.threshold(blur,91,255,cv2.THRESH_BINARY)
        
        contours =  cv2.findContours(thresh_img,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)[-2]
        for c in contours:
            cv2.drawContours(roi2, [c], -1, (0,255,0), 3)

    
        if len(contours)>0:
            try:
                ellipse = cv2.fitEllipse(contours[0])
            except:
                pass
            # get the angle of rotation of the ellipse
            angle = ellipse[2]
            anglecount.append(angle)
            

        ## applying masks 
#         fgmask1 = fgbg.apply(roi1)
#         knnmask1 = knnfgbg.apply(roi1) 
        
#         fgmask2 = fgbg.apply(roi2)
#         knnmask2 = knnfgbg.apply(roi2) 
        
        # contour detection within this space 
        
        # apply edge detection within the rectangle given for the right wrist 
#         edges = cv2.Canny(roi1, 100, 200)
        
#         contours, _ = cv2.findContours(edges, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

        # Draw the contours on the original frame
#         cv2.drawContours(roi2, contours, 4, (0, 255, 0), 2)
        
        
        
#         task1 = frame[320:400,200:280]
        
        # Show the frame with bounding rectangle
        cv2.imshow('frame', frame)
        cv2.imshow("roi1",roi1)
        cv2.imshow("roi2",roi2)
#         cv2.imshow("task1",task1)
        
        ##displays masks which are formed around the left arm (shows the absolute masks) 
#         cv2.imshow("FGBGMASK Left Hand",fgmask1)
#         cv2.imshow("KNNMASK Left Hand",knnmask1)
        
        #displays masks which are formed around the right arm (shows the absolute masks) 
#         cv2.imshow("FGBGMASK RIght Hand",fgmask2)
#         cv2.imshow("KNNMASK right Hand",knnmask2)
        
        
        if cv2.waitKey(5) == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()
